# Figure 5: Reference climate comparison between PMET, CR2MET, MSWEP and ERA5

In [ ]:
# spatial libraries
import shapely.geometry
import rioxarray as rioxr
import geopandas as gpd
import xarray as xr

import pandas as pd
import numpy as np
import os

# plotting libraries
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

os.chdir('/home/rooda/Dropbox/Patagonia/')

## Data

In [ ]:
# Catchment shapefiles
basins = gpd.read_file("zip:////home/rooda/Dropbox/Patagonia/MS2 Results/zenodo/basins_boundaries.zip")
basins = basins[["basin_id", "geometry"]].set_index("basin_id")

data = pd.read_csv("MS2 Results/zenodo/dataset_historical.csv", index_col = "basin_id")
data['basin_name']= data['basin_name'].replace({'Santa Cruz': 'Santa Cruz                          '})
basins = pd.concat([basins, data], axis=1)

In [ ]:
# differences
basins_diff = basins[['basin_name', "basin_zone", "basin_area", "area_RGI6"]].reset_index()
basins_diff["PRSN_CR2MET"] = ((basins.PRSN_CR2MET / basins.PRSN_PMET)-1).values
basins_diff["PRSN_ERA5"]   = ((basins.PRSN_ERA5   / basins.PRSN_PMET)-1).values
basins_diff["PRSN_MSWEP"]  = ((basins.PRSN_MSWEP  / basins.PRSN_PMET)-1).values

basins_diff["T2M_CR2MET"]  = (basins.T2M_CR2MET - basins.T2M_PMET).values
basins_diff["T2M_ERA5"]    = (basins.T2M_ERA5   - basins.T2M_PMET).values
basins_diff["T2M_MSWEP"]   = (basins.T2M_MSWEP  - basins.T2M_PMET).values

basins_diff["PPD_CR2MET"]  = ((basins.PPD_CR2MET / basins.PPD_PMET)-1).values
basins_diff["PPD_ERA5"]    = ((basins.PPD_ERA5   / basins.PPD_PMET)-1).values
basins_diff["PPD_MSWEP"]   = ((basins.PPD_MSWEP  / basins.PPD_PMET)-1).values

basins_diff = basins_diff.replace([np.inf, -np.inf], np.nan)
basins_diff.iloc[:,5:8] = basins_diff.iloc[:,5:8].where(basins_diff.iloc[:,5:8] < 2 , np.nan)
basins_diff["area_RGI6_percent"] = basins_diff["area_RGI6"]/basins_diff.area_RGI6.sum()
basins_diff_m = basins_diff.dropna(subset = ['basin_name'])

In [ ]:
# weighted-means
df_wm = {'PRSN': [(basins_diff.PRSN_CR2MET * basins_diff.area_RGI6).sum()*100 / basins_diff.area_RGI6.sum(), 
               (basins_diff.PRSN_ERA5   * basins_diff.area_RGI6).sum()*100 / basins_diff.area_RGI6.sum(), 
               (basins_diff.PRSN_MSWEP  * basins_diff.area_RGI6).sum()*100 / basins_diff.area_RGI6.sum()], 
      'T2M':  [(basins_diff.T2M_CR2MET  * basins_diff.area_RGI6).sum() / basins_diff.area_RGI6.sum(), 
               (basins_diff.T2M_ERA5    * basins_diff.area_RGI6).sum() / basins_diff.area_RGI6.sum(), 
               (basins_diff.T2M_MSWEP   * basins_diff.area_RGI6).sum() / basins_diff.area_RGI6.sum()], 
      'PPD':  [(basins_diff.PPD_CR2MET  * basins_diff.area_RGI6).sum()*100 / basins_diff.area_RGI6.sum(), 
               (basins_diff.PPD_ERA5    * basins_diff.area_RGI6).sum()*100 / basins_diff.area_RGI6.sum(), 
               (basins_diff.PPD_MSWEP   * basins_diff.area_RGI6).sum()*100 / basins_diff.area_RGI6.sum()]}

df_wm = pd.DataFrame(data=df_wm, index=["CR2MET", "ERA5", "MSWEP"])

## Plot elements

In [ ]:
# basemap for background
geo_map = gpd.read_file("/home/rooda/Dropbox/ArcGIS/Chile/south_america.shp")
geo_map = geo_map[(geo_map.CC == "CI") | (geo_map.CC == "AR")]
geo_map = geo_map.dissolve(by='REGION')
geo_map["geometry"] = geo_map.simplify(0.01)

poly_gdf = shapely.geometry.Polygon([(-76, -55.7), (-76, -40.52), (-68.05, -40.52), (-68.05, -55.7), (-76, -55.8)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[poly_gdf], crs=geo_map.crs)

geo_map = geo_map.clip(poly_gdf)

In [ ]:
# hydrological zone divides
geo_lines = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Basins_Patagonia_ice_divides.shp")

lats = []
lons = []

for feature in geo_lines.geometry:
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        
lat_coords = [-43.2, -45.95,  -46.4,  -47.55,  -49.2,   -50.5,   -52.0, -53.1, -54.8]
lon_coords = [-71.2, -71.7,   -74.5,  -71.7,   -72.2,   -72.3,   -72.1, -71.7, -68.9]
names      = ["PPY", "PCA", "NPI-W", "NPI-E", "SPI-N", "SPI-C", "SPI-S", "GCN", "CDI"]
names  = ['<b>'+x+'</b>' for x in names]

## Plot

In [ ]:
fig = make_subplots(rows=3, cols=3, horizontal_spacing = 0.01, vertical_spacing = 0.02, column_widths = [0.36, 0.36, 0.31], shared_xaxes = True,
                    subplot_titles = ["Mean annual precipitation (PMET)","Mean annual temperature (PMET)", "Relative difference to PMET"],
                    specs=[[{"type": "scattergeo", "rowspan": 3}, {"type": "scattergeo", "rowspan": 3}, {"type": "histogram"}],
                           [          None,                                      None,                  {"type": "histogram"}],
                           [          None,                                      None,                  {"type": "histogram"}]])

cl = px.colors.colorbrewer.RdYlBu
cs = px.colors.colorbrewer.GnBu

## Basemap
for x in range(1,3):
    fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#EAEAF2", "#EAEAF2"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=x)

# Precipitation mean change (a) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()),  locations = basins.index, z = basins['PP_PMET'], 
                            colorscale = [cs[2], cs[4], cs[8]], marker_line_color ='white', marker_line_width=0.1, 
                            zmin = 1000, zmax = 8000, colorbar=dict(len=0.45, x=0.24, y= 0.77, title='Precipitation', ticksuffix = " mm", thickness=20)), row=1, col=1)
fig.add_annotation(text="(a)", font=dict(size=16), x=0.005, y=0.995,  xref = "paper", yref = "paper", showarrow=False)


# Temperature mean change (b) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins['T2M_PMET'], 
                            colorscale=["#ffe9ba", cl[3],  cl[1]], marker_line_color='white', marker_line_width=0.1, 
                            zmin = -1, zmax = 8, colorbar=dict(len=0.45, x=0.59, y= 0.77, title='Temperature', ticksuffix = " ºC", thickness=20)), row=1, col=2)
fig.add_annotation(text="(b)", font=dict(size=16), x=0.367, y=0.995,  xref = "paper", yref = "paper", showarrow=False)

# layout a) and b) ---------------------------------------------------------------------------------------------------------------------
for x in range(1,3):
    ## Add basin and hydrological zone names plus the hydro zone divides
    fig.add_trace(go.Scattergeo(lon = lons, lat = lats, mode = 'lines', line = dict(width = 0.8, color = 'black'), opacity = 0.7, showlegend = False),row=1, col=x)  
    fig.add_trace(go.Scattergeo(lon = lon_coords, lat=lat_coords, mode='text', text=names, textfont=dict(size=12, color = "rgba(0,0,0,0.7)"), showlegend = False),row=1, col=x)
    fig.add_scattergeo(geojson = eval(basins['geometry'].to_json()), locations = basins.index, text = basins['basin_name'], mode = 'text', showlegend = False,
                       textfont=dict(size=11, color = "rgba(0,0,0,0.3)"),row=1, col=x)

fig.update_geos(showframe = True, framewidth = 1,  framecolor = "black", lonaxis_range=[-76, -68], lataxis_range=[-55.8, -40.5], 
                bgcolor = "rgb(255,255,255)", showland = False, showcoastlines = False, showlakes = False)

# Solid precipitation spread (c) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Scatter(y = basins_diff.sort_values("PRSN_CR2MET").PRSN_CR2MET, x = basins_diff.sort_values("PRSN_CR2MET").area_RGI6_percent.cumsum(), 
                         marker_color = cl[1],  name = "CR2MET",  showlegend = False), row=1, col=3)

fig.add_trace(go.Scatter(y = basins_diff.sort_values("PRSN_ERA5").PRSN_ERA5, x = basins_diff.sort_values("PRSN_ERA5").area_RGI6_percent.cumsum(), 
                         marker_color = cl[3], name = "ERA5",  showlegend = False), row=1, col=3)

fig.add_trace(go.Scatter(y = basins_diff.sort_values("PRSN_MSWEP").PRSN_MSWEP, x = basins_diff.sort_values("PRSN_MSWEP").area_RGI6_percent.cumsum(), 
                         marker_color = cl[9], name = "MSWEP",  showlegend = False), row=1, col=3)

fig.update_xaxes(dtick = 0.25, row = 1, col = 3)
fig.update_yaxes(range = [-1,1], title_text="Δ Solid precpitation (%)", side = "right", title_standoff = 0, tickangle = 0, tickformat = ',.0%', row = 1, col = 3)
fig.add_annotation(text="(c)", font=dict(size=16), x=0.03, y=0.90, showarrow=False, row=1, col=3)
fig.add_annotation(text="ERA5 ({:.1f}%)".format(df_wm.PRSN["ERA5"]),   font=dict(size=14, color = cl[3]), x=0.35, y=0.57, textangle=-12, showarrow=False, row=1, col=3)
fig.add_annotation(text="CR2MET ({:.1f}%)".format(df_wm.PRSN["CR2MET"]), font=dict(size=14, color = cl[1]), x=0.35, y=-0.20, textangle=-20, showarrow=False, row=1, col=3)
fig.add_annotation(text="MSWEP ({:.1f}%)".format(df_wm.PRSN["MSWEP"]),  font=dict(size=14, color = cl[9]), x=0.35, y=-0.55, textangle=-5, showarrow=False, row=1, col=3)

# Air temperature (d) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Scatter(y = basins_diff.sort_values("T2M_CR2MET").T2M_CR2MET, x = basins_diff.sort_values("T2M_CR2MET").area_RGI6_percent.cumsum(), 
                         marker_color = cl[1], name = "CR2MET",  showlegend = False), row=2, col=3)

fig.add_trace(go.Scatter(y = basins_diff.sort_values("T2M_ERA5").T2M_ERA5, x = basins_diff.sort_values("T2M_ERA5").area_RGI6_percent.cumsum(), 
                         marker_color = cl[3], name = "ERA5", showlegend = False), row=2, col=3)

fig.add_trace(go.Scatter(y = basins_diff.sort_values("T2M_MSWEP").T2M_MSWEP, x = basins_diff.sort_values("T2M_MSWEP").area_RGI6_percent.cumsum(), 
                         marker_color = cl[9], name = "MSWEP", showlegend = False), row=2, col=3)

fig.update_xaxes(dtick = 0.25, row = 2, col = 3)
fig.update_yaxes(range = [-3,3], title_text="Δ Air temperature (°C)", side = "right", title_standoff = 25, tickangle = 0, row = 2, col = 3)
fig.add_annotation(text="(d)", font=dict(size=16), x=0.03, y=2.5, showarrow=False, row=2, col=3)
fig.add_annotation(text="ERA5 ({:.1f}°C)".format(df_wm.T2M["ERA5"]),   font=dict(size=14, color = cl[3]), x=0.45, y=-1.7, textangle=-5, showarrow=False, row=2, col=3)
fig.add_annotation(text="CR2MET ({:.1f}°C)".format(df_wm.T2M["CR2MET"]), font=dict(size=14, color = cl[1]), x=0.45, y=-0.4, textangle=-15, showarrow=False, row=2, col=3)
fig.add_annotation(text="MSWEP ({:.1f}°C)".format(df_wm.T2M["MSWEP"]),  font=dict(size=14, color = cl[9]), x=0.45, y=0.62, textangle=-12, showarrow=False, row=2, col=3)

# Positive degree-day sum (e) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Scatter(y = basins_diff.sort_values("PPD_CR2MET").PPD_CR2MET, x = basins_diff.sort_values("PPD_CR2MET").area_RGI6_percent.cumsum(), 
                         marker_color = cl[1], name = "CR2MET",  showlegend = False), row=3, col=3)

fig.add_trace(go.Scatter(y = basins_diff.sort_values("PPD_ERA5").PPD_ERA5, x = basins_diff.sort_values("PPD_ERA5").area_RGI6_percent.cumsum(), 
                         marker_color = cl[3], name = "ERA5",  showlegend = False), row=3, col=3)

fig.add_trace(go.Scatter(y = basins_diff.sort_values("PPD_MSWEP").PPD_MSWEP, x = basins_diff.sort_values("PPD_MSWEP").area_RGI6_percent.cumsum(), 
                         marker_color = cl[9], name = "MSWEP",  showlegend = False), row=3, col=3)

fig.update_xaxes(range = [-0.02,1.02], title_text="Probability of exceedance (glacier area; %)", title_standoff = 0, dtick = 0.25, tickangle = 0, tickformat = ',.0%', row = 3, col = 3)
fig.update_yaxes(range = [-1,1], title_text="Δ Positive degree-day sum (%)", side = "right", title_standoff = 0, tickangle = 0, tickformat = ',.0%', row = 3, col = 3)
fig.add_annotation(text="(e)", font=dict(size=16), x=0.03, y=0.90, showarrow=False, row=3, col=3)
fig.add_annotation(text="ERA5 ({:.1f}%)".format(df_wm.PPD["ERA5"]),   font=dict(size=14, color = cl[3]), x=0.4, y=-0.40, textangle=-5, showarrow=False, row=3, col=3)
fig.add_annotation(text="CR2MET ({:.1f}%)".format(df_wm.PPD["CR2MET"]), font=dict(size=14, color = cl[1]), x=0.35, y=0.20, textangle=-12, showarrow=False, row=3, col=3)
fig.add_annotation(text="MSWEP ({:.1f}%)".format(df_wm.PPD["MSWEP"]),  font=dict(size=14, color = cl[9]), x=0.35, y=-0.15, textangle=-9, showarrow=False, row=3, col=3)

# layout c, d and e
fig.update_traces(line = dict(width = 2), opacity = 0.7, row = 1, col = 3)
fig.update_traces(line = dict(width = 2), opacity = 0.7, row = 2, col = 3)
fig.update_traces(line = dict(width = 2), opacity = 0.7, row = 3, col = 3)
fig.update_xaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.8)", zeroline=False, showline = True, linecolor = 'black', linewidth = 1, ticks="outside", mirror=True)
fig.update_yaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.8)", zeroline=True,  showline = True, linecolor = 'black', linewidth = 1, ticks="outside", mirror=True)

# general 
fig.update_layout(autosize = False, template = "seaborn", width = 1000, height = 670, margin = dict(l=10, r=5, b=5, t=30, pad=0, autoexpand=True))
fig.show()

fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_5_climate.png", scale=4)

In [ ]:
"Mean precipitation was greater than 4,000 mm yr-1 over {:.1f}% of the glacier area, and {:.1f}% of the glacier area showed an mean temperature above 0 ºC".format(
    basins[basins["PP_PMET"] > 4000].area_RGI6.sum()*100 / basins.area_RGI6.sum(),
    basins[basins["T2M_PMET"] > 0].area_RGI6.sum()*100 / basins.area_RGI6.sum())

In [ ]:
"In relation to PMET, the glacier area-weighted means of solid precipitation were {:.1f}%, {:.1f}% and {:.1f}%, for CR2MET, ERA5 and MSWEP, respectively".format(
    df_wm.PRSN["CR2MET"], df_wm.PRSN["ERA5"], df_wm.PRSN["MSWEP"]) 

In [ ]:
"In relation to PMET, the glacier area-weighted means of temperature were {:.2f}ºC, {:.2f}ºC and {:.2f}ºC, for CR2MET, ERA5 and MSWEP, respectively".format(
    df_wm.T2M["CR2MET"], df_wm.T2M["ERA5"], df_wm.T2M["MSWEP"]) 

In [ ]:
"In relation to PMET, the glacier area-weighted means of PPD were {:.1f}%, {:.1f}% and {:.1f}%, for CR2MET, ERA5 and MSWEP, respectively".format(
    df_wm.PPD["CR2MET"], df_wm.PPD["ERA5"], df_wm.PPD["MSWEP"]) 